In [1]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

In [2]:
import os
from kaggle_secrets import UserSecretsClient

try:
    GOOGLE_API_KEY = UserSecretsClient().get_secret("GOOGLE_API_KEY")
    os.environ["GOOGLE_API_KEY"] = GOOGLE_API_KEY
    print("✅ Gemini API key setup complete.")
except Exception as e:
    print(
        f"🔑 Authentication Error: Please make sure you have added 'GOOGLE_API_KEY' to your Kaggle secrets. Details: {e}"
    )

✅ Gemini API key setup complete.


In [6]:
from google.adk.agents import Agent, LlmAgent, SequentialAgent, ParallelAgent, LoopAgent
from google.adk.models.google_llm import Gemini
from google.adk.runners import InMemoryRunner, Runner
from google.adk.tools import AgentTool, FunctionTool, google_search
from google.genai import types

from google.adk.sessions import InMemorySessionService

print("✅ ADK components imported successfully.")

✅ ADK components imported successfully.


In [4]:
retry_config=types.HttpRetryOptions(
    attempts=5,  # Maximum retry attempts
    exp_base=7,  # Delay multiplier
    initial_delay=1, # Initial delay before first retry (in seconds)
    http_status_codes=[429, 500, 503, 504] # Retry on these HTTP errors
)

In [15]:
# Define helper functions that will be reused throughout the notebook
async def run_session(
    runner_instance: Runner,
    user_queries: list[str] | str = None,
    session_name: str = "default",
):
    print(f"\n ### Session: {session_name}")

    # Get app name from the Runner
    app_name = runner_instance.app_name

    # Attempt to create a new session or retrieve an existing one
    try:
        session = await session_service.create_session(
            app_name=app_name, user_id=USER_ID, session_id=session_name
        )
    except:
        session = await session_service.get_session(
            app_name=app_name, user_id=USER_ID, session_id=session_name
        )

    # Process queries if provided
    if user_queries:
        # Convert single query to list for uniform processing
        if type(user_queries) == str:
            user_queries = [user_queries]

        # Process each query in the list sequentially
        for query in user_queries:
            print(f"\nUser > {query}")

            # Convert the query string to the ADK Content format
            query = types.Content(role="user", parts=[types.Part(text=query)])

            # Stream the agent's response asynchronously
            async for event in runner_instance.run_async(
                user_id=USER_ID, session_id=session.id, new_message=query
            ):
                # Check if the event contains valid content
                if event.content and event.content.parts:
                    # Filter out empty or "None" responses before printing
                    if (
                        event.content.parts[0].text != "None"
                        and event.content.parts[0].text
                    ):
                        print(event.content.parts[0].text)
    else:
        print("No queries!")


print("✅ Helper functions defined.")

✅ Helper functions defined.


In [8]:
# Research Agent: Its job is to use the google_search tool and present findings.
research_agent = Agent(
    name="OpenDatasetSeeker",
    model=Gemini(
        model="gemini-2.5-flash-lite",
        retry_options=retry_config
    ),
    instruction="""You are an open data research agent. Your job is to use the
    google_search tool to find several open data sources of relevant information on the given topic and present the 
    findings like a list of the datasources along with their description and explaining what exact data can be taken from there""",
    tools=[google_search],
    output_key="dataset_findings",  # The result of this agent will be stored in the session state with this key.
)

print("✅ OpenDatasetSeeker created.")

✅ OpenDatasetSeeker created.


In [9]:
# Summarizer Agent: Its job is to summarize the text it receives.
single_dataset_agent = Agent(
    name="SingleDatasetAgent",
    model=Gemini(
        model="gemini-2.5-flash-lite",
        retry_options=retry_config
    ),
    # The instruction is modified to request a bulleted list for a clear output format.
    instruction="""Your goal is to compose the list of parameters which can be taken from each of the provided datasources 
    in order to get comprehencive data-driven view on the topic. Create a summary of what data is available and from what source""",
    output_key="final_summary",
)

print("✅ SingleDatasetAgent created.")

✅ SingleDatasetAgent created.


In [10]:
# Root Coordinator: Orchestrates the workflow by calling the sub-agents as tools.
root_agent = SequentialAgent(
    name="ResearchCoordinator",
    sub_agents=[research_agent, single_dataset_agent]
)

print("✅ root_agent created.")

✅ root_agent created.


In [11]:
APP_NAME = "default"  # Application
USER_ID = "default"  # User
SESSION = "default"
session_service = InMemorySessionService()
runner = Runner(agent=root_agent, app_name=APP_NAME, session_service=session_service)
print("✅ Runner created.")

✅ Runner created.


In [16]:
response = await run_session(
    runner,
    [
    "What countrues in Europe consume the most electricity?",
        "Which country out of these has the larget GDP?"]
)


 ### Session: default

User > What countrues in Europe consume the most electricity?
When looking at electricity consumption in Europe, it's important to consider both total consumption and consumption per capita.

**Countries with the Highest Total Electricity Consumption (2023):**
*   **Germany:** 515 TWh
*   **France:** 464 TWh
*   **Italy:** 315 TWh
*   **United Kingdom:** 319 billion kWh
*   **Spain:** 256 TWh
*   **Poland:** 145 billion kWh
*   **Sweden:** 129 billion kWh
*   **Norway:** 116 billion kWh
*   **Netherlands:** 107 billion kWh

**Countries with the Highest Electricity Consumption Per Capita (2022/2023):**
*   **Iceland:** 52,920 kWh/year
*   **Norway:** 23,374 kWh/year
*   **Finland:** 14,747 kWh/year
*   **Sweden:** 12,122 kWh/year
*   **Luxembourg:** 11,168 kWh/year

**Key Data Sources for European Electricity Consumption:**

*   **Ember:** This organization provides detailed annual reports on the European electricity sector, including data on electricity generati